In [1]:
# pip install tiktoken -q

In [2]:
# pip install pandas

In [1]:
! python --version

Python 3.8.19


In [4]:
# !pip install langchain-community -q

In [5]:
# pip install langchain -q

In [2]:
import time
import pandas as pd
import json
import os
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate)
from langchain import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.schema import (SystemMessage, HumanMessage, AIMessage)
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.docstore.document import Document

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def recurs_split(x):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=400, 
        chunk_overlap=50
    )
    split_docs = text_splitter.create_documents([x])
    return(split_docs)

In [8]:
#examples

In [4]:
examples = [
{'text': 'Vincent Yen Tek Chiu, a 44-year-old Canadian man from Vancouver, was sentenced to 174 months in federal prison for leading a drug-trafficking organization that smuggled hundreds of pounds of cocaine, heroin, and MDMA into Canada and the United States. The organization used big-rig trucks and encrypted telephones to transport the drugs. Chiu was found guilty by a federal jury in March on multiple counts related to drug distribution and exportation. He was involved in distributing 77 kilograms of cocaine, 8 kilograms of heroin, and 24 kilograms of MDMA, which had a wholesale value of over $3 million. The investigation, led by the FBI, Homeland Security Investigations, and the Royal Canadian Mounted Police, seized over $800,000 in Canadian currency and resulted in prison sentences for several other members of the organization.',
  'output': [
{'head': 'Vincent Yen Tek Chiu',
    'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
   {'head': '44-year-old',
    'head type': 'age',
    'relation': 'age of',
    'tail': 'Vincent Yen Tek Chiu',
    'tail type': 'primary name'},
   {'head': 'Canadian',
    'head type': 'nationality',
    'relation': 'nationality of',
    'tail': 'Vincent Yen Tek Chiu',
    'tail type': 'primary name'},
   {'head': 'Vancouver',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Vincent Yen Tek Chiu',
    'tail type': 'primary name'},
   {'head': '174 months in federal prison',
    'head type': 'sentence',
    'relation': 'sentence of',
    'tail': 'Vincent Yen Tek Chiu',
    'tail type': 'primary name'},
   {'head': 'leading a drug-trafficking organization that smuggled hundreds of pounds of cocaine, heroin, and MDMA into Canada and the United States. Multiple counts related to drug distribution and exportation. Involved in distributing 77 kilograms of cocaine, 8 kilograms of heroin, and 24 kilograms of MDMA',
    'head type': 'crime',
    'relation': 'crime of',
    'tail': 'Vincent Yen Tek Chiu',
    'tail type': 'primary name'},
   {'head': 'March',
    'head type': 'date',
    'relation': 'date of announcing',
    'tail': '174 months in federal prison',
    'tail type': 'sentence'},
   {'head': 'over $3 million',
    'head type': 'amount (value)',
    'relation': 'amount (value) of',
    'tail': 'leading a drug-trafficking organization that smuggled hundreds of pounds of cocaine, heroin, and MDMA into Canada and the United States. Multiple counts related to drug distribution and exportation. Involved in distributing 77 kilograms of cocaine, 8 kilograms of heroin, and 24 kilograms of MDMA ',
    'tail type': 'crime'}]},

{'text': 'On November 26, the Police arrested Santosh Kisku, alias Santosh Dehri, a wanted member of the Communist Party of India-Maoist (CPI-Maoist), during a raid at his house in Hadwadangal village, Dumka District, Jharkhand. Kisku was an area commander for the Tritiya Sammelan Prastuti Committee (TSPC), a splinter group of CPI-Maoist. Additionally, another TSPC cadre, Upendra Yadav from Bhadaibathan village, Latehar district, was also arrested in a raid on November 26. \n On November 28, the Police recovered a cane bomb weighing around 20 kilograms hidden under the ground in Khutwar Forest, Ango Police Station limits, Hazaribagh District, Jharkhand. The bomb was reportedly kept by CPI-Maoist to target Security Forces (SFs).',
  'output': [
{'head': 'Santosh Kisku',
    'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': 'Santosh Dehri',
    'head type': 'alias',
    'relation': 'alias of',
    'tail': 'Santosh Kisku',
    'tail type': 'primary name'},
{'head': 'Tritiya Sammelan Prastuti Committee (TSPC), a splinter group of Communist Party of India-Maoist (CPI-Maoist)',
    'head type': 'Connected Party (Organization)',
    'relation': 'connection with',
    'tail': 'Santosh Kisku',
    'tail type': 'primary name'},
{'head': 'November 26',
    'head type': 'date of arrest',
    'relation': 'date of arrest of',
    'tail': 'Santosh Kisku',
    'tail type': 'primary name'},
{'head': 'Area Commander',
    'head type': 'designation',
    'relation': 'designation of',
    'tail': 'Santosh Kisku',
    'tail type': 'primary name'},
{'head': 'Area Commander',
    'head type': 'designation',
    'relation': 'designation in',
    'tail': 'Tritiya Sammelan Prastuti Committee (TSPC), a splinter group of CPI-Maoist',
    'tail type': 'Connected Party (Organization)'},
{'head': 'Hadwadangal village, Dumka District, Jharkhand',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Santosh Kisku',
    'tail type': 'primary name'},
{'head': 'Upendra Yadav',
    'head type': 'Connected Party (Individual)',
    'relation': 'connection with',
        'tail': 'Santosh Kisku',
    'tail type': 'primary name'},
{'head': 'Bhadaibathan village, Latehar district',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Upendra Yadav',
    'tail type': 'Connected Party (Individual)'}]},

{'text': "Rapid Action Battalion (RAB) members detained two suspected members of the banned militant outfit Ansar Al Islam from Betbaria area in Narail on Wednesday. The arrestees were Md Abu Nayeem, 25, and Shahriar Rana, 22. A team of RAB-6 conducted a drive at Masud Miah's house and detained them. Shahriar Rana was working as the treasurer of Khulna unit of Ansar-al-Islam, according to a RAB press release. The elite force members also seized books on extremism and laptops from their possessions. Later, Rab members handed them over to Narail Sadar police on Wednesday night. A case was filed in this regard on Thursday, and legal action will be taken accordingly, said Mahmudur Rahman, officer-in-charge (Investigation) of Narail Sadar Police Station.",
  'output': [
{'head': 'Md Abu Nayeem',
    'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': '25',
    'head type': 'age',
    'relation': 'age of',
    'tail': 'Md Abu Nayeem',
    'tail type': 'primary name'},
{'head': 'Shahriar Rana',
     'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': '22',
    'head type': 'age',
    'relation': 'age of',
    'tail': 'Shahriar Rana',
    'tail type': 'primary name'},
{'head': 'Ansar Al Islam',
    'head type': 'Connected Party (Organization)',
    'relation': 'connection with',
    'tail': 'Md Abu Nayeem',
    'tail type': 'primary name'}, 
{'head': 'Ansar Al Islam',
    'head type': 'Connected Party (Organization)',
    'relation': 'connection with',
    'tail': 'Shahriar Rana',
    'tail type': 'primary name'},
{'head': 'Treasurer',
    'head type': 'designation',
    'relation': 'designation of',
    'tail': 'Shahriar Rana',
    'tail type': 'primary name'},
{'head': 'Treasurer',
    'head type': 'designation',
    'relation': 'designation in',
    'tail': 'Ansar Al Islam',
    'tail type': 'Connected Party (Organization)'},
{'head': 'Betbaria area in Narail',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Md Abu Nayeem',
    'tail type': 'primary name'},
{'head': 'Betbaria area in Narail',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Shahriar Rana',
    'tail type': 'primary name'},
{'head': 'Masud Miah',
    'head type': 'Connected Party (Individual)',
    'relation': 'connection with',
    'tail': 'Md Abu Nayeem',
    'tail type': 'primary name'},
{'head': 'Masud Miah',
    'head type': 'Connected Party (Individual)',
    'relation': 'connection with',
    'tail': 'Shahriar Rana',
    'tail type': 'primary name'}]},

{'text': "The Counter-Terrorism Department (CTD) arrested six alleged terrorists across Punjab on July 9, 2022. The arrested individuals include Wahid Bhatti, Jameel Ur Rehman, Mohammad Abdullah, Mohsin Khursheed, Imran Khan, and Kashif Ali. Two of the suspects were from Lahore, while four others were from various districts in Punjab. During the arrests, CTD teams recovered arms, ammunition, and explosive material from the suspects' possession. The investigation is currently underway at an undisclosed location.",
  'output': [
{'head': 'Wahid Bhatti',
    'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': 'Jameel Ur Rehman',
     'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': 'Mohammad Abdullah',
     'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': 'Imran Khan',
     'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': 'Mohsin Khursheed',
     'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': 'Kashif Ali',
     'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': 'July 9, 2022',
    'head type': 'date of arrest',
    'relation': 'date of arrest of',
    'tail': 'Wahid Bhatti',
    'tail type': 'primary name'},
{'head': 'July 9, 2022',
    'head type': 'date of arrest',
    'relation': 'date of arrest of',
    'tail': 'Jameel Ur Rehman',
    'tail type': 'primary name'},
{'head': 'July 9, 2022',
    'head type': 'date of arrest',
    'relation': 'date of arrest of',
    'tail': 'Mohammad Abdullah',
    'tail type': 'primary name'},
{'head': 'July 9, 2022',
    'head type': 'date of arrest',
    'relation': 'date of arrest of',
    'tail': 'Imran Khan',
    'tail type': 'primary name'},
{'head': 'July 9, 2022',
    'head type': 'date of arrest',
    'relation': 'date of arrest of',
    'tail': 'Mohsin Khursheed',
    'tail type': 'primary name'},
{'head': 'July 9, 2022',
    'head type': 'date of arrest',
    'relation': 'date of arrest of',
    'tail': 'Kashif Ali',
    'tail type': 'primary name'},
{'head': 'across Punjab',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Wahid Bhatti',
    'tail type': 'primary name'},
{'head': 'across Punjab',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Jameel Ur Rehman',
    'tail type': 'primary name'},
{'head': 'across Punjab',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Mohammad Abdullah',
    'tail type': 'primary name'},
{'head': 'across Punjab',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Imran Khan',
    'tail type': 'primary name'},
{'head': 'across Punjab',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Mohsin Khursheed',
    'tail type': 'primary name'},
{'head': 'across Punjab',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Kashif Ali',
    'tail type': 'primary name'}]},

{'text': 'Shaiful Azman Taha, a 39-year-old director at the Contractor Service Center of the Works Ministry in Pahang, was charged with receiving a bribe of RM42,000 in 2013. The alleged bribery occurred on December 21, 2013, when Shaiful Azman received the money from Hamdan Mat Hasan, a technician at NZH Murni Enterprise, as an inducement to issue a bumiputera contractor certificate for the company. If found guilty, Shaiful Azman faces up to 20 years in prison and a fine of five times the amount of the bribe or RM10,000, whichever is higher. He was arrested on February 8 last year following a complaint lodged with the Malaysian Anti-Corruption Commission (MACC) on January 26. Shaiful Azman was released on bail of RM15,000 in one surety and his case will be mentioned again on February 28 for submission of documents.',
  'output': [
{'head': 'Shaiful Azman Taha',
    'head type': 'primary name',
    'relation': 'none',
    'tail': 'none',
    'tail type': 'none'},
{'head': '39-year-old ',
    'head type': 'age',
    'relation': 'age of',
    'tail': 'Shaiful Azman Taha',
    'tail type': 'primary name'},
{'head': 'Contractor Service Center of the Works Ministry',
    'head type': 'Connected Party (Organization)',
    'relation': 'connection with',
    'tail': 'Shaiful Azman Taha',
    'tail type': 'primary name'}, 
{'head': 'Director',
    'head type': 'designation',
    'relation': 'designation of',
    'tail': 'Shaiful Azman Taha',
    'tail type': 'primary name'},
{'head': 'Director',
    'head type': 'designation',
    'relation': 'designation in',
    'tail': 'Contractor Service Center of the Works Ministry',
    'tail type': 'Connected Party (Organization)'},
{'head': 'Pahang',
    'head type': 'location',
    'relation': 'location of',
    'tail': 'Shaiful Azman Taha',
    'tail type': 'primary name'},
{'head': 'alleged bribery as an inducement to issue a bumiputera contractor certificate for the company',
    'head type': 'crime',
    'relation': 'crime of',
    'tail': 'Shaiful Azman Taha',
    'tail type': 'primary name'},
{'head': 'RM 42,000',
    'head type': 'amount (value)',
    'relation': 'amount (value) of',
    'tail': 'alleged bribery as an inducement to issue a bumiputera contractor certificate for the company',
    'tail type': 'crime'},
{'head': 'December 21, 2013',
    'head type': 'date',
    'relation': 'date of committing',
    'tail': 'alleged bribery as an inducement to issue a bumiputera contractor certificate for the company',
    'tail type': 'crime'},
{'head': 'faces up to 20 years in prison and a fine of five times the amount of the bribe or RM10,000, whichever is higher',
    'head type': 'sentence',
    'relation': 'sentence of',
    'tail': 'Shaiful Azman Taha',
    'tail type': 'primary name'},
{'head': 'February 8',
    'head type': 'date of arrest',
    'relation': 'date of arrest of',
    'tail': 'Shaiful Azman Taha',
    'tail type': 'primary name'},
{'head': 'Hamdan Mat Hasan',
    'head type': 'Connected Party (Individual)',
    'relation': 'connection with',
    'tail': 'Shaiful Azman Taha',
    'tail type': 'primary name'}]}
    ]

In [5]:
#new entities list
entity_types = ['primary name', 'age', 'nationality', 'location', 'sentence', 'crime', 'date', 'amount (value)', 'date of arrest', 'alias', 
                'designation', 'Connected Party (Organization)', 'Connected Party (Individual)']
# entity_types = ['parties involved in transaction', 'Residence location', 'primary name', 'activity location', 'time reported of location', 
#                 'time period of residence', 'is deceased', 'employer name', 'alias', 'Address location', 'deceased date', 'id type', 'martial status', 
#                 'age', 'activity at location', 'time period of address', 'gender', 'transaction amount', 'issued on', 'age of', 'employer type', 
#                 'employment reported on', 'location of', 'marriage date', 'id belongs to', 'relative or closely associated person', 'name of deceased', 
#                 'identification no', 'activity', 'null', 'reported time period', 'organization (terrorist)', 'residence of', 'age reported on date', 
#                 'designation', 'date', 'expiry on', 'employement period', 'is PEP', 'deceased status', 'none', 'location', 'employer of', 'address of', 
#                 'deceased status reported on date', 'marital status reported on date', 'employment period', 'biography_person', 
#                 'organization (government)', 'organization (others)', 'son of']

relation_types = ['none', 'age of', 'nationality of', 'location of', 'sentence of', 'crime of', 'date of announcing', 'amount (value) of', 
                  'date of arrest of', 'alias of', 'designation of', 'designation in', 'connection with']
# relation_types  = ['none', 'citizenship', 'brother of', 'place of birth of', 'from', 'related or closely associated to', 'joint account holder with', 
#                    'gender of', 'spouse of', 'alias of', 'son of', 'nationality', 'native of', 'date of birth of', 'daughter of', 'father of']

In [6]:
print(len(entity_types), len(relation_types))

13 13


In [7]:
#this piece works

system_prompt = PromptTemplate(
    template = """
    You are a top-tier algorithm designed for extracting information in structured formats.
    Your task is to identify the entities and relations requested with the user prompt, from a given text.
    You must generate the output in a JSON containing a list with JSON objects having the following keys: "head", "head_type", "relation", "tail", and "tail_type", as is also specified in the examples.
    The "head" key must contain the text of the extracted entity with one of the types from the provided list in the user prompt. 
    The "head_type" key must contain the type of the extracted head entity which must be one of the types from {entity_types}.
    The "relation" key must contain the type of relation between the "head" and the "tail" which must be one of the relations from {relation_types}.
    The "tail" key must represent the text of an extracted entity which is the tail of the relation, and the "tail_type" key must contain the type of the tail entity from {entity_types}. 
    
    Attempt to extract as many entities and relations as you can. Only use entity and relation type as mentioned. 
    DO NOT RETURN ANY OTHER ENTITY OR RELATION TYPE.
    
    IMPORTANT NOTES:
    - Don't add any explanation and text. 
    - Strictly adhere to formats in examples.
    - If information about any field is not present in the text, return 'None' for those fields. 
    - If information about any field is not present in the text, DO NOT publish any information from the added examples. 
    """,
    input_variables=["entity_types", "relation_types"],
)

system_message_prompt = SystemMessagePromptTemplate(prompt = system_prompt)

human_prompt = PromptTemplate(
    template = """ Based on the following example, extract entities and relations from the provided text.\n\n

    Use only the following entity types, don't return any other entity type:
    # ENTITY TYPES:
    {entity_types}

    Use only the following relation types, don't return any other relation type:
    # RELATION TYPES:
    {relation_types}

    Below are a number of examples of text and their extracted entities and relationshhips.
    {examples}

    For the following text, generate extract entitites and relations as in the provided example.\nText: {text}""",
    input_variables=["entity_types", "relation_types", "examples", "text"],
)

human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

model = ChatOllama(model = "llama3", temperature=0)
chain = LLMChain(llm=model, prompt=chat_prompt)

# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# llm = ChatOllama(model = "llama3", temperature=0)
# chain = prompt | llm

C:\Users\admin\anaconda3\envs\TM_NS\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
#LOAD DATASET
sum_df = pd.read_csv(r'C:\Users\admin\Downloads\NER_KS\LLM_Output_PT_20240626.csv', encoding_errors='replace')['Sum_Prompt_Templates']
# runset20jun = pd.read_csv(r'C:\Users\admin\Downloads\ot_anjali\all_mrgd_ot_gpu.csv')#['sent']

In [32]:
# sum_df = open('LLM_Output_PT_20240626.csv', encoding="cp437", errors='ignore')

In [9]:
sum_df

0      I apologize, but the provided text appears to ...
1      Vincent Yen Tek Chiu, a 44-year-old Canadian m...
2      Thirteen people were arrested on two federal g...
3      Here is a summary of the provided text:\n\nAcc...
4      Unfortunately, the provided text does not cont...
                             ...                        
145    After the Army Public School attack in Decembe...
146    Here is a summary of the provided text:\n\nAft...
147    According to a report published on December 9t...
148    On December 8, 2018, the Counter Terrorism Dep...
149    Here is a summary of the provided text:\n\nIn ...
Name: Sum_Prompt_Templates, Length: 150, dtype: object

In [15]:
f_df = pd.DataFrame()
for i in range(len(sum_df)):
# for i in range(1, 2):
    print(i)
    st_t = time.time()
    recusi = recurs_split(sum_df[i])
    response = chain.run(entity_types = entity_types, relation_types = relation_types, examples = examples, text = recusi)
    et_t = time.time()
    total_time = (et_t-st_t)/60
    print('time taken for model', total_time)
    a = pd.DataFrame([[sum_df[i], response, total_time]], columns = ['sent', 'output_model', 'time taken'])
    f_df = pd.concat([f_df, a])
    if i%10==0:
         nm  = 'ot_ner_ks/llm_ed_'+str(i+1)+'_08july24.csv'
         f_df.to_csv(nm)
    print('for', i, 'concat done')
f_df.to_csv( 'ot_ner_ks/llm_ed_FINAL_08july24.csv')

0
time taken for model 0.18039594093958536
for 0 concat done
1
time taken for model 0.5408810456593831
for 1 concat done
2
time taken for model 0.5924118796984355
for 2 concat done
3
time taken for model 0.27102137804031373
for 3 concat done
4
time taken for model 0.15482171773910522
for 4 concat done
5
time taken for model 0.3967129349708557
for 5 concat done
6
time taken for model 0.33757476806640624
for 6 concat done
7
time taken for model 0.327472726504008
for 7 concat done
8
time taken for model 0.3420081893603007
for 8 concat done
9
time taken for model 0.2860099593798319
for 9 concat done
10
time taken for model 0.26838066975275676
for 10 concat done
11
time taken for model 0.32844268480936684
for 11 concat done
12
time taken for model 0.30416170358657835
for 12 concat done
13
time taken for model 0.31150984366734824
for 13 concat done
14
time taken for model 0.3798672676086426
for 14 concat done
15
time taken for model 0.410213307539622
for 15 concat done
16
time taken for mode

In [14]:
a['output_model'].tolist()

["Here is the extracted entities and relations:\n\n[{'head': 'Vincent Yen Tek Chiu', 'head type': 'primary name', 'relation': 'none', 'tail': 'none', 'tail type': 'none'}, {'head': '44-year-old ', 'head type': 'age', 'relation': 'age of', 'tail': 'Vincent Yen Tek Chiu', 'tail type': 'primary name'}, {'head': 'Canadian man', 'head type': 'nationality', 'relation': 'nationality of', 'tail': 'Vincent Yen Tek Chiu', 'tail type': 'primary name'}, {'head': 'Vancouver', 'head type': 'location', 'relation': 'location of', 'tail': 'Vincent Yen Tek Chiu', 'tail type': 'primary name'}, {'head': 'federal prison', 'head type': 'Connected Party (Organization)', 'relation': 'connection with', 'tail': 'Vincent Yen Tek Chiu', 'tail type': 'primary name'}, {'head': '174 months', 'head type': 'sentence', 'relation': 'sentence of', 'tail': 'Vincent Yen Tek Chiu', 'tail type': 'primary name'}, {'head': 'drug-trafficking organization', 'head type': 'Connected Party (Organization)', 'relation': 'connection w